In [31]:

import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Load datasets
df_1k = pd.read_csv("df_1k.csv")
df_10k = pd.read_csv("df_10k.csv")
df_100k = pd.read_csv("df_100k.csv")


In [32]:

def prepare_data(df):
    X = df.drop(columns=["outcome"])
    y = df["outcome"]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [33]:

def build_model(input_dim, layers=1):
    model = Sequential()
    model.add(Dense(4, input_dim=input_dim, activation='relu'))
    if layers == 2:
        model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [34]:

def train_and_evaluate(X_train, X_test, y_train, y_test, layers=1):
    model = build_model(X_train.shape[1], layers)
    start = time.time()
    history = model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0, validation_data=(X_test, y_test))
    end = time.time()
    train_loss = history.history['loss'][-1]
    val_loss = history.history['val_loss'][-1]
    elapsed_time = end - start
    return train_loss, val_loss, elapsed_time


In [35]:

results = []

for name, df in zip(["1k", "10k", "100k"], [df_1k, df_10k, df_100k]):
    X_train, X_test, y_train, y_test = prepare_data(df)

    # 1 hidden layer
    train_loss_1, val_loss_1, time_1 = train_and_evaluate(X_train, X_test, y_train, y_test, layers=1)

    # 2 hidden layers
    train_loss_2, val_loss_2, time_2 = train_and_evaluate(X_train, X_test, y_train, y_test, layers=2)

    results.append([name, "1 hidden layer", train_loss_1, val_loss_1, time_1])
    results.append([name, "2 hidden layers", train_loss_2, val_loss_2, time_2])

results_df = pd.DataFrame(results, columns=["Data Size", "Configuration", "Training Error", "Validation Error", "Execution Time (s)"])
results_df.to_csv("week12_deep_learning_results.csv", index=False)
results_df


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Data Size    Configuration  Training Error  Validation Error  \
0        1k   1 hidden layer        0.356843          0.404949   
1        1k  2 hidden layers        0.289417          0.309350   
2       10k   1 hidden layer        0.032063          0.030504   
3       10k  2 hidden layers        0.014740          0.012916   
4      100k   1 hidden layer        0.006489          0.006429   
5      100k  2 hidden layers        0.004099          0.004022   

   Execution Time (s)  
0            6.030063  
1            5.594206  
2           20.860074  
3           24.091119  
4          149.848086  
5          169.455612